# Welcome to PYNQ

## Getting Started

To get started using PYNQ, try running the example notebooks in the folders described below. 

* **getting_started**: includes an introduction to using Jupyter notebook with PYNQ, the Python environment, and how to use some basic features of the curernt platform. 

* **common**: contains example notebooks on how to download an overlay, how to set the Zynq clocks, how to execute Linux shell commands, and how to use USB devices.

If other overlays or packages are installed, other folders with example notebooks may also be available in this directory.  


## Documentation

Please see the latest <a href="http://pynq.readthedocs.io">PYNQ Documentation on readthedocs</a>.  


## Support

For questions or support, go to the forum on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## Project webpage

You can find details on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## GitHub

The PYNQ Repository is hosted on github: <a href="https://github.com/Xilinx/PYNQ">PYNQ GitHub Repository</a>.

In [277]:
import findspark
findspark.init()


In [278]:

import pyspark

In [279]:
from pyspark.sql import SparkSession

In [280]:
pyspark.__version__

'2.4.5'

In [281]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config('spark.sql.execution.arrow.fallback.enabled', 'true')\
    .config("spark.sql.execution.arrow.enabled", "true")\
    .master("local[*]")\
    .getOrCreate()


In [282]:
df = spark.read.csv('hdfs://afog-master:9000/part4-projects/resources/benchmarks/dataset-1_converted.csv')
df.show()

+----------+---+---+
|       _c0|_c1|_c2|
+----------+---+---+
|1582999200|  1|109|
|1582999260|  1|100|
|1582999320|  1|104|
|1582999380|  1|108|
|1582999440|  1|105|
|1582999500|  1| 99|
|1582999560|  1| 96|
|1582999620|  1|102|
|1582999680|  1|121|
|1582999740|  1| 99|
|1582999800|  1|109|
|1582999860|  1| 93|
|1582999920|  1|113|
|1582999980|  1|106|
|1583000040|  1|112|
|1583000100|  1|101|
|1583000160|  1|108|
|1583000220|  1|110|
|1583000280|  1|108|
|1583000340|  1|104|
+----------+---+---+
only showing top 20 rows



In [283]:
# Get rid of slotOccupancy
df = df.drop(df.columns[2])
df.show()

+----------+---+
|       _c0|_c1|
+----------+---+
|1582999200|  1|
|1582999260|  1|
|1582999320|  1|
|1582999380|  1|
|1582999440|  1|
|1582999500|  1|
|1582999560|  1|
|1582999620|  1|
|1582999680|  1|
|1582999740|  1|
|1582999800|  1|
|1582999860|  1|
|1582999920|  1|
|1582999980|  1|
|1583000040|  1|
|1583000100|  1|
|1583000160|  1|
|1583000220|  1|
|1583000280|  1|
|1583000340|  1|
+----------+---+
only showing top 20 rows



In [257]:
import os

In [258]:
os.getcwd()

'/home/xilinx/jupyter_notebooks'

In [245]:
from pynq import Overlay, Clocks

print(f'CPU:   {Clocks.cpu_mhz:.6f}MHz')
print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')


CPU:   650.000000MHz
FCLK0: 76.923077MHz


In [204]:
# Use bitstream for PL 
overlay = Overlay(os.getcwd() + "/smart_parking_stream/dpu_ip/dpu.bit")

In [205]:
overlay?

In [206]:
# DPU IP
dpu_ip = overlay.dpu_ip
dpu_ip?

In [207]:
# DMA IP
dma = overlay.axi_dma
dma?

In [208]:
hw_timer = overlay.axi_timer
hw_timer?

In [209]:
#  Shows the Registers we need to access - Can use access via names or direct memory  (via names is easier)
dpu_ip.register_map


RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  max_size = Register(max_size=0),
  max_size_ctrl = Register(max_size_ap_vld=0, RESERVED=0)
}

In [214]:
# Smart Parking Stream

# First signal to set high. Ensures AP_START Signal does not go low after one cycle
# In AXI_STREAM, only setting AP_START enables computations for 1 stream
dpu_ip.register_map.CTRL.AUTO_RESTART = 1
# Computations occur while high
dpu_ip.register_map.CTRL.AP_START = 1

In [215]:
# Check if signals were set and read only registers have values
dpu_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=1, AP_IDLE=0, AP_READY=1, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  max_size = Register(max_size=4294967295),
  max_size_ctrl = Register(max_size_ap_vld=1, RESERVED=0)
}

In [216]:
# Maximum no of elements dpu accepts in input Stream
## This is for a finite size for loop implementation
## While loop implementation can accept unlimited size of stream
max_size = int(dpu_ip.register_map.max_size)
print(max_size)

4294967295


In [217]:
################# Test: Feed a single stream ##################

In [259]:
# Does near zero time Spark to Pandas df conversion. 
# Required features not supported on Armv7 as of yet
# @link for issue I put up 
#       https://issues.apache.org/jira/browse/ARROW-10276
!pip show pyarrow

Name: pyarrow
Version: 0.17.0
Summary: Python library for Apache Arrow
Home-page: https://arrow.apache.org/
Author: Apache Arrow Developers
Author-email: dev@arrow.apache.org
License: Apache License, Version 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy
Required-by: 


In [284]:
import pandas as pd


df_pd = df.toPandas()
npArr = df_pd.to_numpy()

/opt/spark/python/pyspark/sql/dataframe.py:2110: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  module 'pyarrow' has no attribute 'compat'
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [230]:
df_pd.tail

<bound method NDFrame.tail of                _c0 _c1
0       1582999200   1
1       1582999260   1
2       1582999320   1
3       1582999380   1
4       1582999440   1
...            ...  ..
172404  1590628995   1
172405  1590629055   1
172406  1590629115   1
172407  1590629175   1
172408  1590629235   1

[172409 rows x 2 columns]>

In [220]:
# Convert 2D to 1D array
print(npArr.shape)
npArr = npArr.flatten()
print(npArr.shape)

(172409, 2)
(344818,)


In [221]:
import numpy as np
from pynq import allocate

row_length = 2

inStream = allocate(shape=(npArr.size), dtype=np.uint32)

# Simply assigning the list will not work
for i, x in enumerate(npArr):
    inStream[i] = npArr[i]

# Confirm records were transferred
print(inStream)

outStream = allocate(shape=(npArr.size//row_length), dtype=np.uint32)

[1582999200          1 1582999260 ...          1 1590629235          1]


In [222]:
dma.sendchannel.transfer(inStream)
print("Completed transfer inStream")

Completed transfer inStream


In [223]:
dma.recvchannel.transfer(outStream)
print("Completed transfer outStream")

Completed transfer outStream


In [224]:
# Call if TLAST is not set high in HLS code
dma.sendchannel.wait()
print("Completed send Channel wait")

Completed send Channel wait


In [225]:
# Call if TLAST is not set high in HLS code
dma.recvchannel.wait()
print("Completed recv Channel wait")

Completed recv Channel wait


In [226]:
# outStream.flush()
print(outStream)

[104 105 105 ... 105 105 104]


In [160]:
inStream.freebuffer()
print("Freed inStream buffer")


Freed inStream buffer


In [161]:
outStream.freebuffer()
print("Freed outStream buffer")

Freed outStream buffer
